### Importación de librerías

In [ ]:
import cv2
import mediapipe as mp
import numpy as np
import time
import random
import sys
import pickle
import os
import pygame
import ctypes
import threading
import pyttsx3
from PIL import ImageFont, ImageDraw, Image 

### Configuración y rutas

In [ ]:
FILE_MANOS = "../train/modelo_body_red_augmentation.pkl"
FILE_CARA = "../train/modelo_facial_red_augmentation.pkl"

# Rutas de recursos (Fuentes y Sonidos)
BASE_PATH = os.getcwd()
FONT_CUSTOM = os.path.join(BASE_PATH, "fonts", "HelloMissDi.otf")
FONT_CUSTOM2 = os.path.join(BASE_PATH, "fonts", "CandyPlanet.ttf")
FONT_CUSTOM3 = os.path.join(BASE_PATH, "fonts", "MouldyCheese.ttf")
FONT_CUSTOM4 = os.path.join(BASE_PATH, "fonts", "MfLoveSong.ttf")
SOUND_SUCCESS = os.path.join(BASE_PATH, "sounds", "success.mp3") 
SOUND_FAIL = os.path.join(BASE_PATH, "sounds", "error.mp3")

# --- COLORES & ESTILOS (Paleta Neon/Arcade) ---
COLORS = {
    'neon_yellow': (0, 200, 255),  
    'neon_green':  (50, 255, 50),   
    'neon_purple': (255, 0, 255),   
    'neon_blue':   (255, 255, 0),   
    'neon_orange': (0, 165, 255),   
    'red':         (0, 0, 255),
    'light_grey':  (255, 200, 200), 
    'white':       (255, 255, 255),
    'black':       (0, 0, 0)
}

COLORES_ANIMACION = [
    COLORS['neon_yellow'],
    COLORS['neon_green'],
    COLORS['neon_blue'],
    COLORS['neon_orange'],
    COLORS['neon_purple'],
]

# --- 1. Texto para mostrar qué debe hacer el usuario (INSTRUCCIÓN) ---
INSTRUCCIONES = {
    "1_Mano_Der_Arriba": "Levanta la mano derecha",
    "2_Mano_Izq_Arriba": "Levanta la mano izquierda",
    "3_Punos_Cerrados": "Cierra tu puño",
    "4_Pulgar_Arriba": "Pulgar hacia arriba",
    "5_Victoria": "Haz el símbolo de Victoria o Paz",
    "6_Rock": "Haz los cuernos del Rock",
    "7_Llamada": "Haz el gesto de llamar",
    "8_Ok": "Haz el gesto de Ok",
    "1_Ojos_Cerrados": "Cierra los ojos",
    "2_Cabeza_Der": "Mueve la cabeza hacia la derecha",
    "3_Cabeza_Izq": "Mueve la cabeza hacia la izquierda",
    "0_Neutro": "Neutro"
}

# --- 2. Texto para mostrar qué está detectando el sistema (FEEDBACK) ---
DETECCION = {
    "1_Mano_Der_Arriba": "Mano Derecha",
    "2_Mano_Izq_Arriba": "Mano Izquierda",
    "3_Punos_Cerrados": "Puño",
    "4_Pulgar_Arriba": "Pulgar Arriba",
    "5_Victoria": "Victoria",
    "6_Rock": "Rock",
    "7_Llamada": "Llamada",
    "8_Ok": "OK",
    "1_Ojos_Cerrados": "Ojos Cerrados",
    "2_Cabeza_Der": "Cabeza Derecha",
    "3_Cabeza_Izq": "Cabeza Izquierda",
    "0_Neutro": "Neutro"
}

# --- 3. Texto para gramática de error "Tenías que..." (ERROR) ---
ERROR = {
    "1_Mano_Der_Arriba": "levantar la mano derecha",
    "2_Mano_Izq_Arriba": "levantar la mano izquierda",
    "3_Punos_Cerrados": "cerrar el puño",
    "4_Pulgar_Arriba": "levantar el pulgar",
    "5_Victoria": "hacer el símbolo de la paz",
    "6_Rock": "hacer el gesto de Rock",
    "7_Llamada": "hacer el gesto de llamada",
    "8_Ok": "hacer el gesto de Ok",
    "1_Ojos_Cerrados": "cerrar los ojos",
    "2_Cabeza_Der": "mover la cabeza hacia la derecha",
    "3_Cabeza_Izq": "mover la cabeza hacia la izquierda",
    "0_Neutro": "quedarte neutro"
}

### Procesamiento de Landmarks
Funciones para normalizar las coordenadas de los landmarks de manos y rostro, haciéndolos invariantes a la posición en la pantalla.

In [ ]:
def get_features(landmarks):
    """
    Extrae y normaliza las características para el modelo.
    Centra las coordenadas en el promedio (centroide) y escala
    para que la detección sea invariante a la posición y distancia de la cámara.
    """
    coords = np.array([[lm.x, lm.y] for lm in landmarks.landmark])
    centroid = np.mean(coords, axis=0)
    centered = coords - centroid
    max_dist = np.max(np.abs(centered))
    if max_dist > 0: normalized = centered / max_dist
    else: normalized = centered
    return normalized.flatten()

### Clase principal del juego (Un jugador/Multijugador)

#### Clase Jugador

In [ ]:
class Player:
    def __init__(self, pid, name, bounds):
        self.id = pid
        self.name = name
        self.bounds = bounds # (min_x, max_x)
        self.lives = 3
        self.is_eliminated = False
        
        # Detección
        self.current_gesture = "0_Neutro"
        self.last_gesture = "0_Neutro"
        self.consecutive_frames = 0
        self.stable_gesture = "0_Neutro"
        
        # Estado Ronda
        self.round_status = "PLAYING" # PLAYING, PASSED, FAILED
        self.fail_reason = ""
        self.neutral_start_time = None

    def update_gesture(self, raw_gesture):
        if raw_gesture == self.last_gesture:
            self.consecutive_frames += 1
        else:
            self.last_gesture = raw_gesture
            self.consecutive_frames = 0
        
        if self.consecutive_frames > 2:
            self.stable_gesture = raw_gesture
        else:
            self.stable_gesture = "..."

#### Clase Juego

In [ ]:
class SimonGame:
    def __init__(self):
        """
        Inicializa la instancia del juego, cargando modelos, 
        configurando MediaPipe y estableciendo variables de estado iniciales.
        """
        print("--- SIMON SAYS: MEMORY ---")

        # --- Carga de Modelos de Machine Learning ---
        try:
            with open(FILE_MANOS, 'rb') as f: self.model_hand = pickle.load(f)
            with open(FILE_CARA, 'rb') as f: self.model_face = pickle.load(f)
        except: 
            self.model_hand, self.model_face = None, None
            print("Error cargando modelos")

        # Mapeos internos de las predicciones numéricas a etiquetas de texto
        self.map_hand = {0: "0_Neutro", 1: "1_Mano_Der_Arriba", 2: "2_Mano_Izq_Arriba", 3: "3_Punos_Cerrados", 4: "4_Pulgar_Arriba", 5: "5_Victoria", 6: "6_Rock", 7: "7_Llamada", 8: "8_Ok"}
        self.map_face = {0: "0_Neutro", 1: "1_Ojos_Cerrados", 2: "2_Cabeza_Der", 3: "3_Cabeza_Izq"}
        
        # Generación de lista de acciones disponibles filtrando "Neutro"
        self.acciones_disponibles = []
        for l in list(self.map_hand.values()) + list(self.map_face.values()):
            if "Neutro" not in l and l not in self.acciones_disponibles:
                self.acciones_disponibles.append(l)
        
        # --- Inicialización de Audio (Pygame) ---
        pygame.mixer.init()
        try:
            self.sfx_success = pygame.mixer.Sound(SOUND_SUCCESS)
            self.sfx_success.set_volume(0.5)
            self.sfx_fail = pygame.mixer.Sound(SOUND_FAIL)
            self.sfx_fail.set_volume(0.4)
        except: self.sfx_success = None; self.sfx_fail = None

        # --- VOZ (Pyttsx3) ---
        # Inicializar el motor de voz
        self.engine = pyttsx3.init()
        for v in self.engine.getProperty('voices'):
            if "spanish" in v.name.lower() or "es-es" in v.id.lower():
                self.engine.setProperty('voice', v.id); break
        self.engine.setProperty('rate', 145)
 
        # --- Configuración de MediaPipe ---
        self.mp_hands = mp.solutions.hands
        self.hands = self.mp_hands.Hands(max_num_hands=2, min_detection_confidence=0.5)
        self.mp_face = mp.solutions.face_mesh
        self.face_mesh = self.mp_face.FaceMesh(max_num_faces=2, refine_landmarks=True, min_detection_confidence=0.5)
        self.mp_draw = mp.solutions.drawing_utils

        # --- MODIFICACIÓN: Pantalla completa / Resolución Pantalla ---
        self.cap = cv2.VideoCapture(0)
        user32 = ctypes.windll.user32
        self.cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)
        self.cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)

        self.state = "MENU"         # Estado inicial
        self.players = []           # Lista de jugadores
        self.sequence = []          # Secuencia de pasos acumulada
        self.current_step_idx = 0   # Índice actual dentro de la secuencia
        self.timer_start = 0        # Tiempo de inicio
        self.speech_triggered = False
        
        self.tiempo_memorizar = 2.0     # Segundos para ver el nuevo paso
        self.turn_time = 8.0            # Segundos para ejecutar la acción
        self.tiempo_entre_rondas = 4.0  # Pausa tras éxito
        self.tiempo_espera_trampa = 4.0

    def play_sound(self, t):
        """Reproduce un sonido basado en el tipo de evento."""
        if t == "success" and self.sfx_success: self.sfx_success.play()
        elif t == "fail" and self.sfx_fail: self.sfx_fail.play()

    def hablar(self, txt):
        """Ejecuta la voz en un hilo separado para no congelar la cámara."""
        def _speak():
            try:
                if self.engine._inLoop: self.engine.endLoop()
                self.engine.say(texto)
                self.engine.runAndWait()
            except: pass
        thread = threading.Thread(target=_speak)
        thread.start()

    def poner_texto_utf8(self, frame, text, xy, color=(255,255,255), size=30, font_path=None):
        """
        Dibuja texto sobre el frame utilizando la librería Pillow.
        Permite el uso de caracteres UTF-8 (tildes, ñ) y fuentes personalizadas.
        Convierte de BGR (OpenCV) a RGB (PIL) y viceversa.
        """
        # 1. Convertir imagen de OpenCV (BGR) a Pillow (RGB)
        img_pil = Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
        draw = ImageDraw.Draw(img_pil)

        # 2. Convertir el color recibido (BGR) a RGB para que Pillow lo entienda
        color_rgb = color[::-1]
        
        font = None
        
        # 3. Intento de cargar la fuente personalizada
        if font_path: 
            try: font = ImageFont.truetype(font_path, size)
            except: pass
        # 4. Intento de cargar Arial del sistema Windows
        if not font:
            try: font = ImageFont.truetype("arial.ttf", size)
            except: font = ImageFont.load_default()

        # 5. Cálculo de dimensiones para centrado
        bbox = draw.textbbox((0, 0), text, font=font)
        text_w = bbox[2] - bbox[0]
        x, y = xy
        
        # 6. Si x es -1, se centra el texto horizontalmente en la imagen
        if x == -1: x = (frame.shape[1] - text_w) // 2
        
        draw.text((x, y), text, font=font, fill=color_rgb)
        # Retorna la imagen convertida de nuevo a formato OpenCV
        return cv2.cvtColor(np.array(img_pil), cv2.COLOR_RGB2BGR)

    def dibujar_tarjeta_hud(self, frame, titulo, l1, l2, progreso):
        H, W, _ = frame.shape
        card_w, card_h = 600, 250
        x1, y1 = (W - card_w)//2, (H - card_h)//2
        overlay = frame.copy()
        cv2.rectangle(overlay, (x1, y1), (x1+card_w, y1+card_h), (20,20,20), -1)
        cv2.addWeighted(overlay, 0.7, frame, 0.3, 0, frame)
        bw = int(card_w * progreso)
        bc = (0,255,0) if progreso > 0.3 else (0,0,255)
        cv2.rectangle(frame, (x1, y1+card_h-10), (x1+bw, y1+card_h), bc, -1)
        idx = int(time.time()*3)%len(COLORES_ANIMACION)
        frame = self.poner_texto_utf8(frame, titulo, (-1, y1+25), COLORES_ANIMACION[idx], 40, FONT_CUSTOM)
        y_tx = y1+80
        if l2:
            frame = self.poner_texto_utf8(frame, l1, (-1, y_tx), COLORS['white'], 50, FONT_CUSTOM)
            frame = self.poner_texto_utf8(frame, l2, (-1, y_tx+60), COLORS['white'], 50, FONT_CUSTOM)
        else:
            frame = self.poner_texto_utf8(frame, l1, (-1, y_tx+30), COLORS['white'], 60, FONT_CUSTOM)
        return frame

    def draw_ui_original(self, frame, main, sub="", color=(255, 255, 255)):
        """Dibuja la interfaz de usuario estándar (barra superior) usando OpenCV."""
        H, W, _ = frame.shape
        overlay = frame.copy()
        cv2.rectangle(overlay, (0, 0), (W, 110), (0, 0, 0), -1)
        cv2.addWeighted(overlay, 0.6, frame, 0.4, 0, frame)
        cv2.putText(frame, main, (20, 50), cv2.FONT_HERSHEY_SIMPLEX, 1.0, color, 3)
        cv2.putText(frame, sub, (20, 90), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (200, 200, 200), 2)
        p = self.players[0]
        frame = self.poner_texto_utf8(frame, "o " * p.lives, (W - 200, 60), COLORS['red'], 30, FONT_CUSTOM4)
        return frame

    def add_sequence_step(self):
        acc = random.choice(self.acciones_disponibles)
        current_round_progress = len(self.sequence) * 10 
        
        # 1. Definir qué trampas están desbloqueadas según la dificultad
        trampas_posibles = ['modesto']
        if current_round_progress >= 30: trampas_posibles.append('simon_fake')
        if current_round_progress >= 60: trampas_posibles.append('no_name')

        # 2. Decidir si es válida o trampa usando probabilidades
        # Cambia 0.7 a 0.8 si quieres 80% válido, o 0.6 para 60%, etc.
        PROBABILIDAD_VALIDA = 0.7  

        if random.random() < PROBABILIDAD_VALIDA:
            tipo = 'valid'
        else:
            # Si cae en el 30% restante, se elige una trampa al azar de las disponibles
            tipo = random.choice(trampas_posibles)

        # 3. Asignar textos y flags
        if tipo == 'valid': h, tr = "Simón dice:", False
        elif tipo == 'modesto': h, tr = "Modesto dice:", True
        elif tipo == 'simon_fake': h, tr = "Simón:", True
        elif tipo == 'no_name': h, tr = "Dice:", True
        
        self.sequence.append({'action': acc, 'header': h, 'is_trap': tr, 'type': tipo})

    def detectar_y_actualizar(self, frame):
        rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        gestos_temp = {p.id: "0_Neutro" for p in self.players}
        
        rh = self.hands.process(rgb)
        if rh.multi_hand_landmarks and self.model_hand:
            for lm in rh.multi_hand_landmarks:
                self.mp_draw.draw_landmarks(frame, lm, self.mp_hands.HAND_CONNECTIONS)
                cx = np.mean([l.x for l in lm.landmark])
                try:
                    feat = get_features(lm)
                    pred = self.model_hand.predict([feat])[0]
                    prob = np.max(self.model_hand.predict_proba([feat]))
                    if prob > 0.5:
                        label = self.map_hand[pred]
                        if "Neutro" not in label:
                            for p in self.players:
                                if p.bounds[0] <= cx <= p.bounds[1]: gestos_temp[p.id] = label
                except: pass

        rf = self.face_mesh.process(rgb)
        if rf.multi_face_landmarks and self.model_face:
            for lm in rf.multi_face_landmarks:
                cx = np.mean([l.x for l in lm.landmark])
                for p in self.players:
                    if p.bounds[0] <= cx <= p.bounds[1] and gestos_temp[p.id] == "0_Neutro":
                        try:
                            feat = get_features(lm)
                            pred = self.model_face.predict([feat])[0]
                            prob = np.max(self.model_face.predict_proba([feat]))
                            if prob > 0.6:
                                label = self.map_face[pred]
                                if "Neutro" not in label: gestos_temp[p.id] = label
                        except: pass
        for p in self.players: p.update_gesture(gestos_temp[p.id])

    def run(self):
        cv2.namedWindow('SIMON GAME', cv2.WND_PROP_FULLSCREEN)
        cv2.setWindowProperty('SIMON GAME', cv2.WND_PROP_FULLSCREEN, cv2.WINDOW_FULLSCREEN)

        while self.cap.isOpened():
            ret, frame = self.cap.read()
            if not ret: break
            frame = cv2.flip(frame, 1)
            H, W, _ = frame.shape
            
            # --- CÁLCULO DE TAMAÑOS DINÁMICOS ---
            font_title = int(H * 0.12) 
            font_subtitle = int(H * 0.06)
            font_button = int(H * 0.04)
            
            if self.state == "MENU":
                cv2.rectangle(frame, (0,0), (W,H), (10,10,10), -1)
                frame = self.poner_texto_utf8(frame, "SIMON SAYS", (-1, int(H*0.2)), COLORS['neon_purple'], font_title, FONT_CUSTOM)
                frame = self.poner_texto_utf8(frame, "Selecciona una opción:", (-1, int(H*0.45)), COLORS['light_grey'], font_subtitle, FONT_CUSTOM2)
                frame = self.poner_texto_utf8(frame, "1] Un Jugador", (-1, int(H*0.6)), COLORS['white'], font_button, FONT_CUSTOM2)
                frame = self.poner_texto_utf8(frame, "2] Dos Jugadores (VS)", (-1, int(H*0.7)), COLORS['white'], font_button, FONT_CUSTOM2)
                cv2.putText(frame, "Pulsa 'ESC' para salir", (int(W*0.38), int(H*0.9)), cv2.FONT_HERSHEY_SIMPLEX, H*0.001, COLORS['light_grey'], 2)
                
                k = cv2.waitKey(1) & 0xFF
                if k == ord('1'):
                    self.players = [Player(1, "Jugador", (0.0, 1.0))]
                    self.sequence = []
                    self.add_sequence_step()
                    self.state = "SHOW_NEW_STEP"
                    self.timer_start = time.time()
                    self.speech_triggered = False
                    self.current_step_idx = 0
                
                elif k == ord('2'):
                    self.players = [Player(1, "Jugador 1", (0.0, 0.5)), Player(2, "Jugador 2", (0.5, 1.0))]
                    self.state = "CHECK_PLAYERS"
                
                elif k == 27: break

            elif self.state == "CHECK_PLAYERS":
                cv2.line(frame, (W//2, 0), (W//2, H), (255,255,255), 2)
                self.detectar_y_actualizar(frame)
                ready = 0
                for p in self.players:
                    cx = int((p.bounds[0]+p.bounds[1])/2 * W)
                    r = p.stable_gesture != "..."
                    if r: ready += 1
                    txt = "LISTO" if r else "ESPERANDO..."
                    col = COLORS['neon_green'] if r else COLORS['red']
                    frame = self.poner_texto_utf8(frame, f"{p.name}", (cx-100, H//2-60), COLORS['white'], 35, FONT_CUSTOM2)
                    frame = self.poner_texto_utf8(frame, txt, (cx-50, H//2), col, 40, FONT_CUSTOM)
                frame = self.poner_texto_utf8(frame, "[ ESPACIO ]", (-1, H-100), COLORS['light_grey'], 30, FONT_CUSTOM)
                if cv2.waitKey(1) & 0xFF == 32 and ready == 2:
                    self.sequence = []
                    self.add_sequence_step()
                    self.state = "SHOW_NEW_STEP"
                    self.timer_start = time.time()
                    self.speech_triggered = False
                    self.current_step_idx = 0

            elif self.state == "SHOW_NEW_STEP":
                step = self.sequence[-1]
                elapsed = time.time() - self.timer_start
                prog = 1.0 - (elapsed / self.tiempo_memorizar)
                txt = INSTRUCCIONES.get(step['action'], step['action'])
                l1, l2 = "", ""
                for w in txt.split():
                    if len(l1+" "+w)<=18: l1+=(" " if l1 else "")+w
                    else: l2+=(" " if l2 else "")+w
                if not self.speech_triggered:
                    self.hablar(f"{step['header']} {txt}")
                    self.speech_triggered = True
                frame = self.dibujar_tarjeta_hud(frame, step['header'], l1, l2, max(0, prog))
                
                if elapsed > self.tiempo_memorizar:
                    self.state = "WAIT_NEUTRAL"
                    #  Al iniciar nueva ronda global, se resetean los estados de todos
                    for p in self.players: p.round_status = "PLAYING"; p.neutral_start_time = None

            elif self.state == "WAIT_NEUTRAL":
                self.detectar_y_actualizar(frame)
                all_ok = True
                for p in self.players:
                    if not p.is_eliminated and p.stable_gesture != "0_Neutro": all_ok = False
                
                if len(self.players) == 2: cv2.line(frame, (W//2,0), (W//2,H), (200,200,200), 2)
                if len(self.players) == 1: frame = self.draw_ui_original(frame, "VUELVE A NEUTRO", "Preparando...", COLORS['light_grey'])
                else: frame = self.poner_texto_utf8(frame, "POSICION NEUTRA", (-1, H//2), COLORS['light_grey'], 50, FONT_CUSTOM)
                
                if all_ok:
                    self.state = "PLAYER_TURN"
                    self.timer_start = time.time()
                    # Al salir de neutro, se habilita a los jugadores para el SIGUIENTE paso de la cadena
                    for p in self.players:
                        if not p.is_eliminated:
                            p.round_status = "PLAYING"
                            p.fail_reason = ""
                            p.neutral_start_time = None

            elif self.state == "PLAYER_TURN":
                step = self.sequence[self.current_step_idx]
                elapsed = time.time() - self.timer_start
                remaining = self.turn_time - elapsed
                self.detectar_y_actualizar(frame)

                # ================= 1 JUGADOR =================
                if len(self.players) == 1:
                    p = self.players[0]
                    g = p.stable_gesture
                    instr = f"Repite el paso {self.current_step_idx + 1}/{len(self.sequence)}"
                    frame = self.draw_ui_original(frame, instr, f"Tiempo: {remaining:.1f}s")
                    frame = self.poner_texto_utf8(frame, f"Detectado: {DETECCION.get(g,g)}", (20, H-30), COLORS['neon_green'], 20, FONT_CUSTOM)

                    round_passed, round_failed = False, False
                    if step['is_trap']:
                        if g == "0_Neutro":
                            if p.neutral_start_time is None: p.neutral_start_time = time.time()
                            if time.time() - p.neutral_start_time >= self.tiempo_espera_trampa: round_passed = True
                        elif g != "...":
                            round_failed = True
                            p.fail_reason = "¡Era Modesto!" if step.get('type') == 'modesto' else "¡Trampa! No te muevas."
                        elif remaining <= 0: round_passed = True
                    else:
                        if g == step['action']: round_passed = True
                        elif g != "0_Neutro" and g != "...":
                            round_failed = True
                            p.fail_reason = f"Hiciste {DETECCION.get(g,g)}, era {ERROR.get(step['action'])}"
                        elif remaining <= 0:
                            round_failed = True
                            p.fail_reason = "¡Se acabó el tiempo!"

                    if round_failed:
                        self.play_sound("fail")
                        p.lives -= 1
                        self.state = "LIFE_LOST" if p.lives > 0 else "GAME_OVER"
                        self.timer_start = time.time()
                    elif round_passed:
                        self.play_sound("success")
                        cv2.circle(frame, (W-50, H-50), 40, COLORS['neon_green'], -1)
                        self.current_step_idx += 1
                        if self.current_step_idx < len(self.sequence): self.state = "WAIT_NEUTRAL"
                        else: self.state = "SUCCESS_SEQUENCE"; self.timer_start = time.time()

                # ================= 2 JUGADORES =================
                else:
                    overlay = frame.copy()
                    cv2.rectangle(overlay, (0, 0), (W, 110), (0, 0, 0), -1)
                    cv2.addWeighted(overlay, 0.6, frame, 0.4, 0, frame)
                    
                    txt_paso = f"Repite el paso {self.current_step_idx + 1}/{len(self.sequence)}"
                    txt_tiempo = f"Tiempo: {remaining:.1f}s"
                    cv2.putText(frame, txt_paso, (20, 50), cv2.FONT_HERSHEY_SIMPLEX, 1.0, COLORS['light_grey'], 3)
                    cv2.putText(frame, txt_tiempo, (20, 90), cv2.FONT_HERSHEY_SIMPLEX, 0.7, COLORS['white'], 2)
                    
                    cv2.line(frame, (W//2, 70), (W//2, H), (200,200,200), 2)
                    
                    for p in self.players:
                        if p.is_eliminated:
                            x_min = int(p.bounds[0] * W); x_max = int(p.bounds[1] * W)
                            ov = frame.copy()
                            cv2.rectangle(ov, (x_min, 70), (x_max, H), (0,0,50), -1)
                            cv2.addWeighted(ov, 0.7, frame, 0.3, 0, frame)
                            continue

                        cx = int((p.bounds[0]+p.bounds[1])/2 * W)
                        frame = self.poner_texto_utf8(frame, "o "*p.lives, (cx-40, 135), COLORS['red'], 30, FONT_CUSTOM4)
                        
                        g = p.stable_gesture
                        frame = self.poner_texto_utf8(frame, DETECCION.get(g,g), (cx-50, H-40), COLORS['light_grey'], 20, FONT_CUSTOM)

                        x_min = int(p.bounds[0] * W); x_max = int(p.bounds[1] * W)

                        # --- FEEDBACK VISUAL Y BLOQUEO DE LÓGICA SI YA TERMINÓ ---
                        if p.round_status == "PASSED":
                            ov = frame.copy()
                            cv2.rectangle(ov, (x_min, 70), (x_max, H), (0,255,0), -1)
                            cv2.addWeighted(ov, 0.3, frame, 0.7, 0, frame)
                            continue 

                        elif p.round_status == "FAILED":
                            ov = frame.copy()
                            cv2.rectangle(ov, (x_min, 70), (x_max, H), (0,0,255), -1)
                            cv2.addWeighted(ov, 0.3, frame, 0.7, 0, frame)
                            frame = self.poner_texto_utf8(frame, p.fail_reason, (cx-80, H//2), COLORS['white'], 30, FONT_CUSTOM)
                            continue 
                        
                        # --- LÓGICA DE JUEGO (SOLO SI PLAYING) ---
                        if step['is_trap']:
                            if g != "0_Neutro" and g != "...":
                                p.round_status = "FAILED"; p.fail_reason = "¡Te moviste!"
                                p.lives -= 1; self.play_sound("fail")
                            elif remaining <= 0:
                                p.round_status = "PASSED"   
                        else:
                            if g == step['action']:
                                p.round_status = "PASSED"   
                                self.play_sound("success")
                            elif g != "0_Neutro" and g != "...":
                                p.round_status = "FAILED"; p.fail_reason = "¡Incorrecto!"
                                p.lives -= 1; self.play_sound("fail")
                            elif remaining <= 0:
                                p.round_status = "FAILED"; p.fail_reason = "¡Tiempo!"
                                p.lives -= 1; self.play_sound("fail")

                    # Verificar si TODOS han terminado (FUERA DEL BUCLE FOR)
                    activos = [px for px in self.players if not px.is_eliminated]
                    jugando = [px for px in activos if px.round_status == "PLAYING"]

                    if len(jugando) == 0:
                        time.sleep(0.5)
                        vivos = [px for px in self.players if px.lives > 0]
                        
                        if len(vivos) == 0: self.state = "GAME_OVER"
                        elif len(vivos) == 1 and len(self.players) > 1:
                             # Asegurar eliminación del perdedor visualmente
                             for px in self.players: 
                                 if px.lives <= 0: px.is_eliminated = True
                             self.state = "GAME_OVER"
                        else:
                            self.current_step_idx += 1
                            if self.current_step_idx < len(self.sequence):
                                self.state = "WAIT_NEUTRAL"
                            else:
                                self.state = "SUCCESS_SEQUENCE"
                                self.timer_start = time.time()

            # --- PANTALLAS DE ESTADO ---
            elif self.state == "LIFE_LOST":
                overlay = frame.copy()
                cv2.rectangle(overlay, (0, 0), (W, H), (0, 0, 100), -1)
                cv2.addWeighted(overlay, 0.6, frame, 0.4, 0, frame)
                p = self.players[0]
                
                # Texto de vidas
                if p.lives == 1:
                    frame = self.poner_texto_utf8(frame, f"¡CUIDADO! TE QUEDA {p.lives} VIDA", (-1, H//2-100), COLORS['white'], 30, FONT_CUSTOM2)
                else:
                    frame = self.poner_texto_utf8(frame, f"¡CUIDADO! TE QUEDAN {p.lives} VIDAS", (-1, H//2-100), COLORS['white'], 30, FONT_CUSTOM2)
                
                # --- Lógica para dividir el fail_reason ---
                reason_l1, reason_l2 = "", ""
                # Ajusta el '35' si quieres que la línea sea más larga o más corta
                limit_chars = 35 
                
                for w in p.fail_reason.split():
                    if len(reason_l1 + " " + w) <= limit_chars:
                        reason_l1 += (" " if reason_l1 else "") + w
                    else:
                        reason_l2 += (" " if reason_l2 else "") + w
                
                # Renderizar línea 1
                frame = self.poner_texto_utf8(frame, reason_l1, (-1, H//2), COLORS['white'], 35, FONT_CUSTOM)
                
                # Renderizar línea 2 un poco más abajo (+50 px)
                if reason_l2:
                    frame = self.poner_texto_utf8(frame, reason_l2, (-1, H//2 + 50), COLORS['white'], 35, FONT_CUSTOM)
                # ------------------------------------------

                if time.time() - self.timer_start > 3.0:
                    self.add_sequence_step(); self.state = "SHOW_NEW_STEP"; self.speech_triggered = False; self.timer_start = time.time(); self.current_step_idx = 0    
                    
            elif self.state == "SUCCESS_SEQUENCE":
                overlay = frame.copy()
                cv2.rectangle(overlay, (0,0), (W,H), (0,50,0), -1)
                cv2.addWeighted(overlay, 0.6, frame, 0.4, 0, frame)
                frame = self.poner_texto_utf8(frame, "¡RONDA COMPLETADA!", (-1, H//2), COLORS['white'], 60, FONT_CUSTOM)
                if time.time() - self.timer_start > self.tiempo_entre_rondas:
                    self.add_sequence_step(); self.state = "SHOW_NEW_STEP"; self.current_step_idx = 0; self.timer_start = time.time(); self.speech_triggered = False

            elif self.state == "GAME_OVER":
                overlay = frame.copy()
                cv2.rectangle(overlay, (0,0), (W,H), (0,0,50), -1)
                cv2.addWeighted(overlay, 0.7, frame, 0.3, 0, frame)
                txt = "GAME OVER"
                if len(self.players) > 1:
                    vivos = [p for p in self.players if not p.is_eliminated]
                    txt = f"¡GANADOR: {vivos[0].name}!" if vivos else "¡EMPATE / GAME OVER!"
                frame = self.poner_texto_utf8(frame, txt, (-1, H//2), COLORS['white'], 60, FONT_CUSTOM)
                cv2.putText(frame, "R: Reiniciar   |   ESC: Salir", (W//2 - 175, H - 100), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 255), 2)    
                    
                if cv2.waitKey(1) & 0xFF == ord('r'):
                    self.sequence = []
                    self.state = "MENU"

                elif cv2.waitKey(1) & 0xFF == 27: break

            cv2.imshow('SIMON GAME', frame)
            if cv2.waitKey(1) & 0xFF == 27: break
        
        self.cap.release()
        cv2.destroyAllWindows()
        pygame.quit()


### Iniciar Juego

In [ ]:
if __name__ == "__main__":
    game = SimonGame()
    game.run()